In [ ]:
%pip install pandas

In [ ]:
import pandas as pd

# Load each CSV file
movies = pd.read_csv("/home/babsdevsys/ml-32m/movies.csv")
ratings = pd.read_csv("/home/babsdevsys/ml-32m/ratings.csv")
tags = pd.read_csv("/home/babsdevsys/ml-32m/tags.csv")
links = pd.read_csv("/home/babsdevsys/ml-32m/links.csv")

# Check the shape and column names
print("Movies:", movies.shape, movies.columns)
print("Ratings:", ratings.shape, ratings.columns)
print("Tags:", tags.shape, tags.columns)
print("Links:", links.shape, links.columns)

In [ ]:
# Preview datasets
print(movies.head())
print(ratings.head())
print(tags.head())
print(links.head())

In [6]:
import pandas as pd
import os

# Paths for input and output datasets
INPUT_PATH = "/home/babsdevsys/ml-32m/"
OUTPUT_PATH = "/home/babsdevsys/movie_recommendations/loader/sequelize/data/"

# Load the datasets
movies = pd.read_csv(os.path.join(INPUT_PATH, "movies.csv"))
ratings = pd.read_csv(os.path.join(INPUT_PATH, "ratings.csv"))
tags = pd.read_csv(os.path.join(INPUT_PATH, "tags.csv"))
links = pd.read_csv(os.path.join(INPUT_PATH, "links.csv"))

# Rename columns to match the desired schema
movies.rename(columns={"movieId": "movie_id", "title": "title", "genres": "genres"}, inplace=True)
ratings.rename(columns={"userId": "user_id", "movieId": "movie_id", "rating": "rating", "timestamp": "timestamp"}, inplace=True)
tags.rename(columns={"userId": "user_id", "movieId": "movie_id", "tag": "tag", "timestamp": "timestamp"}, inplace=True)
links.rename(columns={"movieId": "movie_id", "imdbId": "imdb_id", "tmdbId": "tmdb_id"}, inplace=True)

# Convert the timestamp columns to a human-readable format
ratings["timestamp"] = pd.to_datetime(ratings["timestamp"], unit="s")  # Convert from UNIX seconds
tags["timestamp"] = pd.to_datetime(tags["timestamp"], unit="s")        # Convert from UNIX seconds

# Preview the datasets after renaming and conversion
print("Movies Preview:\n", movies.head())
print("Ratings Preview:\n", ratings.head())
print("Tags Preview:\n", tags.head())
print("Links Preview:\n", links.head())

# Save the cleaned datasets
os.makedirs(OUTPUT_PATH, exist_ok=True)  # Ensure the output directory exists
movies.to_csv(os.path.join(OUTPUT_PATH, "movies_clean.csv"), index=False)
ratings.to_csv(os.path.join(OUTPUT_PATH, "ratings_clean.csv"), index=False)
tags.to_csv(os.path.join(OUTPUT_PATH, "tags_clean.csv"), index=False)
links.to_csv(os.path.join(OUTPUT_PATH, "links_clean.csv"), index=False)

print("Datasets have been cleaned and saved to:", OUTPUT_PATH)


In [ ]:
import pandas as pd

movies = pd.read_csv("/home/babsdevsys/movie_recommendations/loader/sequelize/data/movies_clean.csv")
ratings = pd.read_csv("/home/babsdevsys/movie_recommendations/loader/sequelize/data/ratings_clean.csv")
tags = pd.read_csv("/home/babsdevsys/movie_recommendations/loader/sequelize/data/tags_clean.csv")
links = pd.read_csv("/home/babsdevsys/movie_recommendations/loader/sequelize/data/links_clean.csv")

# Check the shape and column names
print("Movies:\n", movies.shape, movies.columns)
print("Ratings:\n", ratings.shape, ratings.columns)
print("Tags:\n", tags.shape, tags.columns)
print("Links:\n", links.shape, links.columns)

In [5]:
print("Movies Preview:\n", movies.head())

Movies Preview:
    movie_id                               title  \
0         1                    Toy Story (1995)   
1         2                      Jumanji (1995)   
2         3             Grumpier Old Men (1995)   
3         4            Waiting to Exhale (1995)   
4         5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
